In [4]:
%load_ext autoreload

%autoreload 2

In [5]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ZDW-CjxKfvQGOAAVRnKZqWbooDklr05s' -O '/kaggle/working/train.csv'

--2024-11-15 08:51:45--  https://docs.google.com/uc?export=download&id=1ZDW-CjxKfvQGOAAVRnKZqWbooDklr05s
Resolving docs.google.com (docs.google.com)... 173.194.193.102, 173.194.193.138, 173.194.193.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.193.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ZDW-CjxKfvQGOAAVRnKZqWbooDklr05s&export=download [following]
--2024-11-15 08:51:45--  https://drive.usercontent.google.com/download?id=1ZDW-CjxKfvQGOAAVRnKZqWbooDklr05s&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.183.132, 2607:f8b0:4001:c64::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.183.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11300729 (11M) [application/octet-stream]
Saving to: '/kaggle/working/train.csv'

/kaggle/working/tra 100%[===================>]  10.7

In [6]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1xTytXJ6_7ndIBUsDKNkzIbdeGVo8XmF0' -O '/kaggle/working/test.csv'

--2024-11-15 08:51:50--  https://docs.google.com/uc?export=download&id=1xTytXJ6_7ndIBUsDKNkzIbdeGVo8XmF0
Resolving docs.google.com (docs.google.com)... 173.194.193.113, 173.194.193.101, 173.194.193.138, ...
Connecting to docs.google.com (docs.google.com)|173.194.193.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1xTytXJ6_7ndIBUsDKNkzIbdeGVo8XmF0&export=download [following]
--2024-11-15 08:51:50--  https://drive.usercontent.google.com/download?id=1xTytXJ6_7ndIBUsDKNkzIbdeGVo8XmF0&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.183.132, 2607:f8b0:4001:c64::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.183.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2849900 (2.7M) [application/octet-stream]
Saving to: '/kaggle/working/test.csv'

/kaggle/working/tes 100%[===================>]   2.72

In [7]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1gy9iGkUWlDLmWvt6fl0ZZCY5-75pGP6Y' -O '/kaggle/working/sample_submission.csv'

--2024-11-15 08:51:54--  https://docs.google.com/uc?export=download&id=1gy9iGkUWlDLmWvt6fl0ZZCY5-75pGP6Y
Resolving docs.google.com (docs.google.com)... 74.125.126.138, 74.125.126.139, 74.125.126.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.126.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1gy9iGkUWlDLmWvt6fl0ZZCY5-75pGP6Y&export=download [following]
--2024-11-15 08:51:54--  https://drive.usercontent.google.com/download?id=1gy9iGkUWlDLmWvt6fl0ZZCY5-75pGP6Y&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.183.132, 2607:f8b0:4001:c64::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.183.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86237 (84K) [application/octet-stream]
Saving to: '/kaggle/working/sample_submission.csv'

/kaggle/working/sam 100%[===================>] 

# Import Libraries

In [8]:
%pip install transformers ipywidgets -qqq

Note: you may need to restart the kernel to use updated packages.


In [32]:
import sys
# sys.path.append('/kaggle/input/ods/pytorch/default/1/')


import os



import pandas as pd

from torch.utils.data import DataLoader

from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder



# from dataset import *

# from model import *
# from model_2 import *

# from trainer import Trainer



import random

import numpy as np

import torch



# torch.manual_seed(42)

torch.cuda.is_available()



def set_seed(seed):

    random.seed(seed)

    np.random.seed(seed)

    torch.manual_seed(seed)

    torch.cuda.manual_seed(seed)

    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.

    torch.backends.cudnn.deterministic = True

    torch.backends.cudnn.benchmark = False



# Set the seed

set_seed(100)

In [6]:
#model

from typing import Dict

import torch
from transformers import AutoModel, BertForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch.nn.functional as F
from torch import Tensor
import random
import numpy as np
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(100)

def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]



class ModelForClassification(torch.nn.Module):

    def __init__(self, model_path: str, config: Dict):
        # Set the seed
        set_seed(100)        
        super(ModelForClassification, self).__init__()

        self.model_name = model_path
        self.config = config
        self.n_classes = config['num_classes']
        self.dropout_rate = config['dropout_rate']
        self.automodel = config.get('automodel', True)
        if self.automodel:
            self.bert = AutoModel.from_pretrained(self.model_name)
        else:
            self.bert = XLMRobertaModel.from_pretrained(self.model_name)
        # self.bert = BertForSequenceClassification.from_pretrained(self.model_name, num_labels=32)
        freeze_bert = config.get('freeze_bert', True)
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False     
        self.pre_classifier = torch.nn.Linear(self.bert.config.hidden_size, 768)
        # self.pre_classifier = torch.nn.Linear(32, 200)
        self.dropout = torch.nn.Dropout(self.dropout_rate)
        self.classifier = torch.nn.Linear(768, self.n_classes)
        self.relu = torch.nn.ReLU()

    def forward(self, input_ids=None, attention_mask=None, batch_dict=None, targets=None):
        # # if input_ids is not None and attention_mask is not None:
        # if is_bert:
        #     output = self.bert(
        #         input_ids=input_ids,
        #         attention_mask=attention_mask
        #     )
        #     hidden_state = output[0] # get last hidden state
        #     hidden_state = hidden_state[:, 0] # get CLS token
        #     hidden_state = self.pre_classifier(hidden_state)
        #     hidden_state = self.relu(hidden_state)
        #     hidden_state = self.dropout(hidden_state)
        #     output = self.classifier(hidden_state)
        #     return output
        # # elif batch_dict is not None:
        # else:
        model_output = self.bert(
            input_ids=batch_dict.get(input_ids),
            attention_mask=batch_dict.get(attention_mask),
        )
        hidden_state = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        # hidden_state = model_output.pooler_output
        # hidden_state = torch.nn.functional.normalize(hidden_state)
        hidden_state = self.pre_classifier(hidden_state)
        hidden_state = self.relu(hidden_state)
        hidden_state = self.dropout(hidden_state)
        output = self.classifier(hidden_state)
        return output

        # hidden_state = output.logits
        # return logits
        
        


In [7]:
# import torch.nn.functional as F
# from torch import Tensor
# from transformers import XLMRobertaTokenizer, XLMRobertaModel


# def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
#     last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
#     return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# input_texts = [
#   'query: How does a corporate website differ from a business card website?',
#   'query: Где был создан первый троллейбус?',
#   'passage: The first trolleybus was created in Germany by engineer Werner von Siemens, probably influenced by the idea of his brother, Dr. Wilhelm Siemens, who lived in England, expressed on May 18, 1881 at the twenty-second meeting of the Royal Scientific Society. The electrical circuit was carried out by an eight-wheeled cart (Kontaktwagen) rolling along two parallel contact wires. The wires were located quite close to each other, and in strong winds they often overlapped, which led to short circuits. An experimental trolleybus line with a length of 540 m (591 yards), opened by Siemens & Halske in the Berlin suburb of Halensee, operated from April 29 to June 13, 1882.',
#   'passage: Корпоративный сайт — содержит полную информацию о компании-владельце, услугах/продукции, событиях в жизни компании. Отличается от сайта-визитки и представительского сайта полнотой представленной информации, зачастую содержит различные функциональные инструменты для работы с контентом (поиск и фильтры, календари событий, фотогалереи, корпоративные блоги, форумы). Может быть интегрирован с внутренними информационными системами компании-владельца (КИС, CRM, бухгалтерскими системами). Может содержать закрытые разделы для тех или иных групп пользователей — сотрудников, дилеров, контрагентов и пр.',
# ]

# tokenizer = XLMRobertaTokenizer.from_pretrained('d0rj/e5-large-en-ru', use_cache=False)
# model = XLMRobertaModel.from_pretrained('d0rj/e5-large-en-ru', use_cache=False)

# batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

# outputs = model(**batch_dict)
# embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# embeddings = F.normalize(embeddings, p=2, dim=1)
# scores = (embeddings[:2] @ embeddings[2:].T) * 100
# print(scores.tolist())
# # [[68.59542846679688, 81.75910949707031], [80.36100769042969, 64.77748107910156]]


In [8]:
# dataset

import torch
from torch.utils.data import Dataset

import random
import numpy as np
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(100)

class FiveDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_seq_len, bert=False, sentence=False):
        # Set the seed
        set_seed(100)
        self.data = dataframe
        self.text = dataframe['text'].tolist()
        self.targets = None
        if 'rate' in dataframe:
            self.targets = dataframe['rate'].tolist()
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.is_bert = bert
        self.is_sentence = sentence

    def __getitem__(self, index):
        text = str(self.text[index])
        text = ' '.join(text.split())

        if self.is_bert:
            inputs = self.tokenizer.encode_plus(
                text,
                None,
                add_special_tokens=True,
                max_length=self.max_seq_len,
                pad_to_max_length=True,
                return_token_type_ids=True,
                truncation=True
            )
            ids = inputs['input_ids']
            mask = inputs['attention_mask']
    
            if self.targets is not None:
                return {
                    'ids': torch.tensor(ids, dtype=torch.long),
                    'mask': torch.tensor(mask, dtype=torch.long),
                    'targets': torch.tensor(self.targets[index], dtype=torch.long)
                }
            else:
                return {
                    'ids': torch.tensor(ids, dtype=torch.long),
                    'mask': torch.tensor(mask, dtype=torch.long),
                    }
                
        elif self.is_sentence:
            inputs = self.tokenizer(text, max_length=self.max_seq_len,  pad_to_max_length=True, truncation=True, return_tensors='pt')
            ids = inputs['input_ids']
            mask = inputs['attention_mask']

            if self.targets is not None:
                return {
                    'input_ids': torch.tensor(ids, dtype=torch.long),
                    'attention_mask': torch.tensor(mask, dtype=torch.long),
                    'targets': torch.tensor(self.targets[index], dtype=torch.long)
                }
            else:
                return {
                    'input_ids': torch.tensor(ids, dtype=torch.long),
                    'attention_mask': torch.tensor(mask, dtype=torch.long),
                    }
        else:
            raise ValueError
            
        
            

    def __len__(self) -> int:
        return len(self.text)


In [9]:
from typing import Dict

import torch
from numpy import asarray
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, AdamW
# from tqdm.notebook import tqdm
from tqdm import tqdm

from sklearn.metrics import f1_score

import random
import numpy as np
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(100)

# from model import ModelForClassification


class Trainer:
    def __init__(self, config: Dict, weights):
        # Set the seed
        set_seed(100)
        self.device = config['device']
        self.config = config
        self.n_epochs = config['n_epochs']
        self.optimizer = None
        # self.opt_fn = lambda model: Adam(
        #     model.parameters(),
        #     config['lr'],
        #     weight_decay=config.get('weight_decay', 0)
        # )
        self.opt_fn = lambda model: AdamW(
            model.parameters(),
            lr=config['lr'],
            weight_decay=config.get('weight_decay', 0)
        )
        self.model = None
        self.history = None
        # self.loss_fn = CrossEntropyLoss()
        # Define the loss function with class weights
        weights = weights.to(self.device)
        self.loss_fn = torch.nn.CrossEntropyLoss(weight=weights)

        self.verbose = config.get('verbose', True)

    def fit(self, model, train_dataloader, val_dataloader):
        # self.model = model.to(self.device)
        self.model = torch.nn.DataParallel(model, device_ids = [0,1]).to(self.device)
        self.optimizer = self.opt_fn(model)
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'val_acc': [],
            'val_f1': []
        }

        for epoch in range(self.n_epochs):
            print(f"Epoch {epoch + 1}/{self.n_epochs}")
            train_info = self.train_epoch(train_dataloader)
            val_info = self.val_epoch(val_dataloader)
            self.history['train_loss'].extend(train_info['loss'])
            self.history['val_loss'].extend([val_info['loss']])
            self.history['val_acc'].extend([val_info['acc']])
            self.history['val_f1'].extend([val_info['f1']])
            # print mean and std train loss
            print(f"Train Loss, mean and std: {asarray(train_info['loss']).mean():.3}, {asarray(train_info['loss']).std():.3}")
            print(f"Val loss: {val_info['loss']}")
            # print val loss, acc and f1
            print(f"Val F1 score: {val_info['f1']:.3}")
            self.save(f'/kaggle/working/model_epoch{epoch + 1}')
        return self.model.eval()

    def train_epoch(self, train_dataloader):
        self.model.train()
        losses = []
        if self.verbose:
            train_dataloader = tqdm(train_dataloader)
        for batch in train_dataloader:
            # ids = batch['ids'].to(self.device, dtype=torch.long)
            # mask = batch['mask'].to(self.device, dtype=torch.long)
            # targets = batch['targets'].to(self.device, dtype=torch.long)
            ids = batch['input_ids'].to(self.device, dtype=torch.long)
            mask = batch['attention_mask'].to(self.device, dtype=torch.long)
            targets = batch['targets'].to(self.device, dtype=torch.long)

            outputs = self.model(ids, mask)
            # encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')
            # outputs = self.model(ids, mask)
            # outputs = self.model(batch_dict=batch)
            loss = self.loss_fn(outputs, targets)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_val = loss.item()
            if self.verbose:
                train_dataloader.set_description(f"Loss={loss_val:.3}")
            losses.append(loss_val)
        return {'loss': losses}

    def val_epoch(self, val_dataloader):
        self.model.eval()
        all_logits = []
        all_labels = []
        if self.verbose:
            val_dataloader = tqdm(val_dataloader)
        with torch.no_grad():
            for batch in val_dataloader:
                ids = batch['input_ids'].to(self.device, dtype=torch.long)
                mask = batch['attention_mask'].to(self.device, dtype=torch.long)
                targets = batch['targets'].to(self.device, dtype=torch.long)
                # ids = batch['ids'].to(self.device, dtype=torch.long)
                # mask = batch['mask'].to(self.device, dtype=torch.long)
                # targets = batch['targets'].to(self.device, dtype=torch.long)
                outputs = self.model(ids, mask)
                # outputs = self.model(batch_dict=batch)
                all_logits.append(outputs)
                all_labels.append(targets)
        all_labels = torch.cat(all_labels).to(self.device)
        all_logits = torch.cat(all_logits).to(self.device)
        loss = self.loss_fn(all_logits, all_labels).item()
        acc = (all_logits.argmax(1) == all_labels).float().mean().item()
        # print(acc)
        weighted_f1 = f1_score(all_labels.cpu().numpy(), all_logits.argmax(1).cpu().numpy(), average='weighted')
        if self.verbose:
            val_dataloader.set_description(f"Loss={loss:.3}; Acc:{acc:.3}")
        return {
            'acc': acc,
            'loss': loss,
            'f1': weighted_f1
        }

    def predict(self, test_dataloader):
        if not self.model:
            raise RuntimeError("You should train the model first")
        self.model.eval()
        predictions = []
        with torch.no_grad():
            for batch in test_dataloader:
                ids = batch['input_ids'].to(self.device, dtype=torch.long)
                mask = batch['attention_mask'].to(self.device, dtype=torch.long)
                # ids = batch['ids'].to(self.device, dtype=torch.long)
                # mask = batch['mask'].to(self.device, dtype=torch.long)
                outputs = self.model(ids, mask)
                # outputs = self.model(batch_dict=batch)
                predictions.extend(outputs.argmax(1).tolist())
        return asarray(predictions)

    def save(self, path: str):
        if self.model.module is None:
            raise RuntimeError("You should train the model first")
        checkpoint = {
            "config": self.model.module.config,
            "trainer_config": self.config,
            "model_name": self.model.module.model_name,
            "model_state_dict": self.model.module.state_dict()
        }
        torch.save(checkpoint, path)

    @classmethod
    def load(cls, path: str):
        ckpt = torch.load(path)
        keys = ["config", "trainer_config", "model_state_dict"]
        for key in keys:
            if key not in ckpt:
                raise RuntimeError(f"Missing key {key} in checkpoint")
        new_model = ModelForClassification(
            ckpt['model_name'],
            ckpt["config"]
        )
        new_model.load_state_dict(ckpt["model_state_dict"])
        new_trainer = cls(ckpt["trainer_config"])
        new_trainer.model = new_model
        # new_trainer.model.to(new_trainer.device)
        new_trainer.model = torch.nn.DataParallel(new_model, device_ids = [0,1]).to(self.device)
        return new_trainer


### gpt code

In [10]:
import torch
from transformers import AutoModel, XLMRobertaTokenizer, XLMRobertaModel
from torch import nn
from typing import Dict
import random
import numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(100)

def average_pool(last_hidden_states: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

class ModelForClassification(nn.Module):

    def __init__(self, model_path: str, config: Dict):
        super(ModelForClassification, self).__init__()
        set_seed(100)
        self.model_name = model_path
        self.config = config
        self.n_classes = config['num_classes']
        self.dropout_rate = config['dropout_rate']
        self.bert = XLMRobertaModel.from_pretrained(self.model_name)
        
        # Freeze BERT layers
        for param in self.bert.parameters():
            param.requires_grad = False
        
        self.pre_classifier = nn.Linear(self.bert.config.hidden_size, 768)
        self.dropout = nn.Dropout(self.dropout_rate)
        self.classifier = nn.Linear(768, self.n_classes)
        self.relu = nn.ReLU()

    def forward(self, input_ids=None, attention_mask=None, batch_dict=None, targets=None):
        model_output = self.bert(
            input_ids=batch_dict.get('input_ids'),
            attention_mask=batch_dict.get('attention_mask'),
        )
        hidden_state = average_pool(model_output.last_hidden_state, batch_dict['attention_mask'])
        hidden_state = self.pre_classifier(hidden_state)
        hidden_state = self.relu(hidden_state)
        hidden_state = self.dropout(hidden_state)
        output = self.classifier(hidden_state)
        return output

# Example Dataset class
class FiveDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_seq_len):
        set_seed(100)
        self.data = dataframe
        self.text = dataframe['text'].tolist()
        self.targets = dataframe['rate'].tolist() if 'rate' in dataframe else None
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __getitem__(self, index):
        text = str(self.text[index])
        text = ' '.join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_seq_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        if self.targets is not None:
            return {
                'input_ids': torch.tensor(ids, dtype=torch.long),
                'attention_mask': torch.tensor(mask, dtype=torch.long),
                'targets': torch.tensor(self.targets[index], dtype=torch.long)
            }
        else:
            return {
                'input_ids': torch.tensor(ids, dtype=torch.long),
                'attention_mask': torch.tensor(mask, dtype=torch.long),
            }

    def __len__(self) -> int:
        return len(self.text)

# Example Trainer class
class Trainer:
    def __init__(self, config: Dict, weights):
        set_seed(100)
        self.device = config['device']
        self.config = config
        self.n_epochs = config['n_epochs']
        self.optimizer = None
        self.opt_fn = lambda model: AdamW(
            model.parameters(),
            lr=config['lr'],
            weight_decay=config.get('weight_decay', 0)
        )
        self.model = None
        self.history = None
        weights = weights.to(self.device)
        self.loss_fn = nn.CrossEntropyLoss(weight=weights)
        self.verbose = config.get('verbose', True)

    def fit(self, model, train_dataloader, val_dataloader):
        self.model = nn.DataParallel(model, device_ids=[0, 1]).to(self.device)
        self.optimizer = self.opt_fn(model)
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'val_acc': [],
            'val_f1': []
        }

        for epoch in range(self.n_epochs):
            print(f"Epoch {epoch + 1}/{self.n_epochs}")
            train_info = self.train_epoch(train_dataloader)
            val_info = self.val_epoch(val_dataloader)
            self.history['train_loss'].extend(train_info['loss'])
            self.history['val_loss'].extend([val_info['loss']])
            self.history['val_acc'].extend([val_info['acc']])
            self.history['val_f1'].extend([val_info['f1']])
            print(f"Train Loss, mean and std: {np.asarray(train_info['loss']).mean():.3}, {np.asarray(train_info['loss']).std():.3}")
            print(f"Val loss: {val_info['loss']}")
            print(f"Val F1 score: {val_info['f1']:.3}")
            self.save(f'/kaggle/working/model_epoch{epoch + 1}')
        return self.model.eval()

    def train_epoch(self, train_dataloader):
        self.model.train()
        losses = []
        if self.verbose:
            train_dataloader = tqdm(train_dataloader)
        for batch in train_dataloader:
            ids = batch['input_ids'].to(self.device, dtype=torch.long)
            mask = batch['attention_mask'].to(self.device, dtype=torch.long)
            targets = batch['targets'].to(self.device, dtype=torch.long)
            outputs = self.model(batch_dict={'input_ids': ids, 'attention_mask': mask})
            loss = self.loss_fn(outputs, targets)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_val = loss.item()
            if self.verbose:
                train_dataloader.set_description(f"Loss={loss_val:.3}")
            losses.append(loss_val)
        return {'loss': losses}

    def val_epoch(self, val_dataloader):
        self.model.eval()
        all_logits = []
        all_labels = []
        if self.verbose:
            val_dataloader = tqdm(val_dataloader)
        with torch.no_grad():
            for batch in val_dataloader:
                ids = batch['input_ids'].to(self.device, dtype=torch.long)
                mask = batch['attention_mask'].to(self.device, dtype=torch.long)
                targets = batch['targets'].to(self.device, dtype=torch.long)
                outputs = self.model(batch_dict={'input_ids': ids, 'attention_mask': mask})
                all_logits.append(outputs)
                all_labels.append(targets)
        all_labels = torch.cat(all_labels).to(self.device)
        all_logits = torch.cat(all_logits).to(self.device)
        loss = self.loss_fn(all_logits, all_labels).item()
        acc = (all_logits.argmax(1) == all_labels).float().mean().item()
        weighted_f1 = f1_score(all_labels.cpu().numpy(), all_logits.argmax(1).cpu().numpy(), average='weighted')
        if self.verbose:
            val_dataloader.set_description(f"Loss={loss:.3}; Acc:{acc:.3}")
        return {
            'acc': acc,
            'loss': loss,
            'f1': weighted_f1
        }

    def predict(self, test_dataloader):
        if not self.model:
            raise RuntimeError("You should train the model first")
        self.model.eval()
        predictions = []
        with torch.no_grad():
            for batch in test_dataloader:
                ids = batch['input_ids'].to(self.device, dtype=torch.long)
                mask = batch['attention_mask'].to(self.device, dtype=torch.long)
                outputs = self.model(batch_dict={'input_ids': ids, 'attention_mask': mask})
                predictions.extend(outputs.argmax(1).tolist())
        return np.asarray(predictions)

    def save(self, path: str):
        if self.model.module is None:
            raise RuntimeError("You should train the model first")
        checkpoint = {
            "config": self.model.module.config,
            "trainer_config": self.config,
            "model_name": self.model.module.model_name,
            "model_state_dict": self.model.module.state_dict()
        }
        torch.save(checkpoint, path)

    @classmethod
    def load(cls, path: str):
        ckpt = torch.load(path)
        keys = ["config", "trainer_config", "model_state_dict"]
        for key in keys:
            if key not in ckpt:
                raise RuntimeError(f"Missing key {key} in checkpoint")
        new_model = ModelForClassification(
            ckpt['model_name'],
            ckpt["config"]
        )
        new_model.load_state_dict(ckpt["model_state_dict"])
        new_trainer = cls(ckpt["trainer_config"])
        new_trainer.model = new_model
        new_trainer.model = torch.nn.DataParallel(new_model, device_ids=[0, 1]).to(new_trainer.device)
        return new_trainer

In [2]:
# /kaggle/input/ods-task2-data/test.csv
# PATH = "/kaggle/input/ods-task2-data/"
PATH = "/kaggle/working"

MAX_LEN = 128

# BATCH_SIZE = 64

BATCH_SIZE = 128

# BATCH_SIZE = 32

# Loading data

In [12]:
import os
import pandas as pd

train_data = pd.read_csv(os.path.join(PATH, "train.csv"))

test_data = pd.read_csv(os.path.join(PATH, "test.csv"))



train_data.head()

,rate,text
0,4,Очень понравилось. Были в начале марта с соба...
1,5,В целом магазин устраивает.\nАссортимент позво...
2,5,"Очень хорошо что открылась 5 ка, теперь не над..."
3,3,Пятёрочка громко объявила о том как она заботи...
4,3,"Тесно, вечная сутолока, между рядами трудно ра..."


# Label encoding

In [16]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()



train_data.rate = le.fit_transform(train_data.rate)

train_data.head()

,rate,text
0,3,Очень понравилось. Были в начале марта с соба...
1,4,В целом магазин устраивает.\nАссортимент позво...
2,4,"Очень хорошо что открылась 5 ка, теперь не над..."
3,2,Пятёрочка громко объявила о том как она заботи...
4,2,"Тесно, вечная сутолока, между рядами трудно ра..."


In [17]:
unique_train = train_data.groupby(by=['text'], dropna=False)['rate'].agg(lambda x: pd.Series.mode(x)[0]).reset_index()



unique_train['rate'] = unique_train['rate'].astype('int64')

unique_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46929 entries, 0 to 46928
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    46929 non-null  object
 1   rate    46929 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 733.4+ KB


In [18]:
# discard rows where 'cleaned_text' is duplicated
import re

def clean_text(text):
    letters_and_spaces = re.sub(r'[^a-zA-Zа-яА-Яё0-9]', ' ', text)
    single_space = re.sub(r'\s+', ' ', letters_and_spaces)
    return single_space.lower()

train_data['cleaned_text'] = train_data['text'].apply(clean_text)
# text = 'I am 123 years old!'

train_data_no_duplicates = train_data.drop_duplicates(subset='cleaned_text', keep=False)
len(train_data_no_duplicates)

45535

# Train Test split

In [ ]:
# train_split, val_split = train_test_split(

#     train_data,

#     test_size=0.8,

#     random_state=100,

#     stratify=train_data.rate

#     )

In [144]:
# # prompt: plot two hists, rate of train and rate of val. use same plot. use probability



# import matplotlib.pyplot as plt



# plt.hist(train_split['rate'], density=True, alpha=0.5, label='train')

# plt.hist(val_split['rate'], density=True, alpha=0.5, label='val')

# plt.xlabel('Rate')

# plt.ylabel('Probability')

# plt.title('Rate Distribution (Train vs. Validation)')

# plt.legend(loc='upper right')

# plt.show()


# Loading tokenizer from pretrained

In [145]:
# tokenizer = AutoTokenizer.from_pretrained(

#     "cointegrated/rubert-tiny2", truncation=True, do_lower_case=True)

# Creating datasets and dataloaders

In [146]:
# train_dataset = FiveDataset(train_split, tokenizer, MAX_LEN)

# val_dataset = FiveDataset(val_split, tokenizer, MAX_LEN)

# test_dataset = FiveDataset(test_data, tokenizer, MAX_LEN)

# Loading pretrained model from Huggingface

In [ ]:
tracker = {} # pairs name: {train_loss: ..., val_loss: ..., etc}


In [148]:
import pickle
from sklearn.utils.class_weight import compute_class_weight
from transformers import XLMRobertaTokenizer, XLMRobertaModel


# trainer_config = {

#     "lr": 3e-5,

#     "n_epochs": 5,

#     "weight_decay": 1e-4,

#     "batch_size": BATCH_SIZE,

#     "device": "cuda" if torch.cuda.is_available() else "cpu",

#     "seed": 100,

# }



def train_and_validate(model_name, trainer_config, model_config, experiment_name):
    
    # experiment_name = experiment_name.upper()
    
    trainer_string = f"lr {trainer_config['lr']}, wd {trainer_config['weight_decay']}, ep {trainer_config['n_epochs']}"
    
    trainer_string += f", batch {trainer_config['batch_size']}"
    
    experiment_name += f", {trainer_string}"
    
    experiment_name = experiment_name.upper()
    
    
    
    # create dataloaders
    
    set_seed(100)
    
    rng = torch.Generator()
    
    rng.manual_seed(100)
    
    is_automodel = trainer_config.get('automodel')

    if is_automodel:    
        tokenizer = AutoTokenizer.from_pretrained(
        
        model_name, truncation=True, do_lower_case=True)

    else:
        tokenizer = XLMRobertaTokenizer.from_pretrained(
        
        model_name, truncation=True, do_lower_case=True, use_cache=False)
    
    BATCH_SIZE = trainer_config['batch_size']
    print('train size ', len(train_split))  
    
    
    train_dataset = FiveDataset(train_split, tokenizer, MAX_LEN, bert=trainer_config.get('is_bert', False), sentence = not trainer_config.get('is_bert', False) )
    
    val_dataset = FiveDataset(val_split, tokenizer, MAX_LEN, bert=trainer_config.get('is_bert', False), sentence = not trainer_config.get('is_bert', False) )
    
    test_dataset = FiveDataset(test_data, tokenizer, MAX_LEN, bert=trainer_config.get('is_bert', False), sentence = not trainer_config.get('is_bert', False) )
    
    class_weights = compute_class_weight('balanced', classes=np.unique(train_split['rate']), y=train_split['rate'])
    class_weights = torch.tensor(class_weights, dtype=torch.float)
    
    
    train_params = {"batch_size": BATCH_SIZE,
    
                  "shuffle": True,
    
                  "num_workers": 0,
    
                  "generator": rng
    
                  }
    
    
    
    test_params = {"batch_size": BATCH_SIZE,
    
           "shuffle": False,
    
           "num_workers": 0
    
           }
    
    
    
    
    
    train_dataloader = DataLoader(train_dataset, **train_params)
    
    val_dataloader = DataLoader(val_dataset, **test_params)
    
    test_dataloader = DataLoader(test_dataset, **test_params)
    
    
    
    # get model, trainer
    
    model = ModelForClassification(
    
    model_name,
    
    config=model_config
    
    )
    
    trainer = Trainer(trainer_config, weights=class_weights)
    
    
    
    print(experiment_name)
    
    print('-'*30)
    
    
    
    trainer.fit(model, train_dataloader, val_dataloader)
    
    
    
    tracker[experiment_name] = trainer.history
    
    with open('tracker.pkl', 'wb') as f:
    
      pickle.dump(tracker, f)
    
    
    
    plt.plot(trainer.history['train_loss'], label='train_loss')
    
    # plt.plot(t.history['val_loss'], label='val_loss')
    
    plt.xlabel('Epoch')
    
    plt.ylabel('Loss')
    
    plt.title(experiment_name + ' Train Loss Over Epochs')
    
    plt.legend()
    
    plt.show()
    
    
    
    plt.plot(trainer.history['val_loss'], label='val_loss')
    
    plt.xlabel('Epoch')
    
    plt.ylabel('Loss')
    
    plt.title(experiment_name + ' Val Loss Over Epochs')
    
    plt.legend()
    
    plt.show()
    
    
    
    plt.plot(trainer.history['val_f1'], label='val_f1')
    
    plt.xlabel('Epoch')
    
    plt.ylabel('F1 weighted')
    
    plt.title(experiment_name + ' Val F1 score Over Epochs')
    
    plt.legend()
    
    plt.show()
    
    
    
    return trainer, test_dataloader


### from gpt

In [16]:
import pickle
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoTokenizer, XLMRobertaTokenizer
import torch
import numpy as np
import matplotlib.pyplot as plt

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def train_and_validate(model_name, trainer_config, model_config, experiment_name):
    # Set the seed
    set_seed(trainer_config['seed'])
    
    # Create experiment name
    trainer_string = f"lr {trainer_config['lr']}, wd {trainer_config['weight_decay']}, ep {trainer_config['n_epochs']}"
    trainer_string += f", batch {trainer_config['batch_size']}"
    experiment_name += f", {trainer_string}"
    experiment_name = experiment_name.upper()
    
    # Create dataloaders
    rng = torch.Generator()
    rng.manual_seed(trainer_config['seed'])
    
    tokenizer = XLMRobertaTokenizer.from_pretrained(model_name, truncation=True, do_lower_case=True, use_cache=False)
    
    BATCH_SIZE = trainer_config['batch_size']
    print('train size ', len(train_split))  
    
    train_dataset = FiveDataset(train_split, tokenizer, MAX_LEN)
    val_dataset = FiveDataset(val_split, tokenizer, MAX_LEN)
    test_dataset = FiveDataset(test_data, tokenizer, MAX_LEN)
    # train_dataset = FiveDataset(train_split, tokenizer, MAX_LEN, bert=trainer_config.get('is_bert', False), sentence=not trainer_config.get('is_bert', False))
    # val_dataset = FiveDataset(val_split, tokenizer, MAX_LEN, bert=trainer_config.get('is_bert', False), sentence=not trainer_config.get('is_bert', False))
    # test_dataset = FiveDataset(test_data, tokenizer, MAX_LEN, bert=trainer_config.get('is_bert', False), sentence=not trainer_config.get('is_bert', False))
    
    class_weights = compute_class_weight('balanced', classes=np.unique(train_split['rate']), y=train_split['rate'])
    class_weights = torch.tensor(class_weights, dtype=torch.float)
    
    train_params = {"batch_size": BATCH_SIZE, "shuffle": True, "num_workers": 0, "generator": rng}
    test_params = {"batch_size": BATCH_SIZE, "shuffle": False, "num_workers": 0}
    
    train_dataloader = DataLoader(train_dataset, **train_params)
    val_dataloader = DataLoader(val_dataset, **test_params)
    test_dataloader = DataLoader(test_dataset, **test_params)
    
    # Get model and trainer
    model = ModelForClassification(model_name, config=model_config)
    trainer = Trainer(trainer_config, weights=class_weights)
    
    print(experiment_name)
    print('-'*30)
    
    trainer.fit(model, train_dataloader, val_dataloader)
    
    tracker[experiment_name] = trainer.history
    with open('tracker.pkl', 'wb') as f:
        pickle.dump(tracker, f)
    
    plt.plot(trainer.history['train_loss'], label='train_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(experiment_name + ' Train Loss Over Epochs')
    plt.legend()
    plt.show()
    
    plt.plot(trainer.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(experiment_name + ' Val Loss Over Epochs')
    plt.legend()
    plt.show()
    
    plt.plot(trainer.history['val_f1'], label='val_f1')
    plt.xlabel('Epoch')
    plt.ylabel('F1 weighted')
    plt.title(experiment_name + ' Val F1 score Over Epochs')
    plt.legend()
    plt.show()
    
    return trainer, test_dataloader

# Creating Trainer object and fitting the model

In [17]:
import gc

gc.collect()

torch.cuda.empty_cache()

### from gpt

In [159]:
train_data_no_duplicates_shuffled_sample = train_data_no_duplicates.sample(frac=1, random_state=100)[:10000]
train_split, val_split = train_test_split(
    train_data_no_duplicates_shuffled_sample,
    test_size=0.9,
    random_state=100,
    stratify=train_data_no_duplicates_shuffled_sample.rate
)

experiment_name = "unique train 90% e5 cleaned dups weighed loss drop 0.3"

model_config = {
    "num_classes": 5,
    "dropout_rate": 0.1,
    "freeze_bert": True,
}

model_name = "d0rj/e5-large-en-ru"

trainer_config = {
    "lr": 1e-3,
    "n_epochs": 10,
    "weight_decay": 1e-4,
    "batch_size": 128,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "seed": 100,
    "automodel": False,
    "is_bert": False
}

trainer, test_dataloader = train_and_validate(model_name, trainer_config, model_config, experiment_name)

train size  1000
UNIQUE TRAIN 90% E5 CLEANED DUPS WEIGHED LOSS DROP 0.3, LR 0.001, WD 0.0001, EP 10, BATCH 128
------------------------------
Epoch 1/10


  0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 71/71 [01:36<00:00,  1.36s/it]


Train Loss, mean and std: 1.93, 0.408
Val loss: 1.627962589263916
Val F1 score: 0.357
Epoch 2/10


  0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 71/71 [01:36<00:00,  1.35s/it]


Train Loss, mean and std: 1.46, 0.0504
Val loss: 1.4101347923278809
Val F1 score: 0.152
Epoch 3/10


  0%|          | 0/8 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
 61%|██████    | 43/71 [00:59<00:38,  1.37s/it]


KeyboardInterrupt: 

In [129]:
task_en = "Classify the customer review for a store into highly negative, negative, neutral, positive, highly positive sentiment"
task_ru = "Классифицируй отзыв покупателя на магазин по тональности: сильно негативный, негативный, нейтральный, положительный, сильно положительный"

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

train_data['text_with_instruct'] = train_data['text'].apply(lambda x: get_detailed_instruct(task_en, x))
test_data['text_with_instruct'] = test_data['text'].apply(lambda x: get_detailed_instruct(task_en, x))
train_data_no_duplicates['text_with_instruct'] = train_data_no_duplicates['text'].apply(lambda x: get_detailed_instruct(task_en, x))
train_data.text_with_instruct[5]

/tmp/ipykernel_30/906901613.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data_no_duplicates['text_with_instruct'] = train_data_no_duplicates['text'].apply(lambda x: get_detailed_instruct(task_en, x))


'Instruct: Classify the customer review for a store into highly negative, negative, neutral, positive, highly positive sentiment\nQuery: Магазин в пешей доступности. После ремонта и реконструкции стал намного лучше и уютнее. Радует появившаяся пекарня и выпечка.'

In [133]:
train_data_no_duplicates.head()

,rate,text,cleaned_text,text_with_instruct
0,3,Очень понравилось. Были в начале марта с соба...,очень понравилось были в начале марта с собако...,Instruct: Classify the customer review for a s...
1,4,В целом магазин устраивает.\nАссортимент позво...,в целом магазин устраивает ассортимент позволя...,Instruct: Classify the customer review for a s...
2,4,"Очень хорошо что открылась 5 ка, теперь не над...",очень хорошо что открылась 5 ка теперь не надо...,Instruct: Classify the customer review for a s...
3,2,Пятёрочка громко объявила о том как она заботи...,пятёрочка громко объявила о том как она заботи...,Instruct: Classify the customer review for a s...
4,2,"Тесно, вечная сутолока, между рядами трудно ра...",тесно вечная сутолока между рядами трудно разо...,Instruct: Classify the customer review for a s...


In [42]:
import warnings

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def average_pool(last_hidden_states: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


def pool(hidden_state, mask, pooling_method="cls"):
    if pooling_method == "mean":
        s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
        d = mask.sum(axis=1, keepdim=True).float()
        return s / d
    elif pooling_method == "cls":
        return hidden_state[:, 0]


# Set the seed
set_seed(42)
model_name = "sismetanin/sbert-ru-sentiment-rureviews"
# Load AutoModel from huggingface model repository
# tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
# model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")
# tokenizer = AutoTokenizer.from_pretrained("ai-forever/ru-en-RoSBERTa")
# model = AutoModel.from_pretrained("ai-forever/ru-en-RoSBERTa")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# tokenizer = AutoTokenizer.from_pretrained("deepvk/USER-bge-m3")
# model = AutoModel.from_pretrained("deepvk/USER-bge-m3")
# tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-large-instruct")
# model = AutoModel.from_pretrained("intfloat/multilingual-e5-large-instruct")

model.eval()

# Check if multiple GPUs are available and use DataParallel
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model, device_ids=[0, 1])

model.to(device)

# Create dataset and dataloader
class FiveDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_seq_len):
        self.data = dataframe
        self.text = dataframe['text'].tolist()
        # self.text = dataframe['text_with_instruct'].tolist()
        self.targets = dataframe['rate'].tolist() if 'rate' in dataframe else None
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __getitem__(self, index):
        text = str(self.text[index])
        # text = ' '.join(text.split())
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_seq_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        if self.targets is not None:
            return {
                'input_ids': torch.tensor(ids, dtype=torch.long),
                'attention_mask': torch.tensor(mask, dtype=torch.long),
                'targets': torch.tensor(self.targets[index], dtype=torch.long)
            }
        else:
            return {
                'input_ids': torch.tensor(ids, dtype=torch.long),
                'attention_mask': torch.tensor(mask, dtype=torch.long),
            }

    def __len__(self) -> int:
        return len(self.text)

# Assuming train_split, val_split, and test_data are already defined
# train_dataset = FiveDataset(train_data_no_duplicates, tokenizer, MAX_LEN)
train_dataset = FiveDataset(train_data, tokenizer, MAX_LEN)
test_dataset = FiveDataset(test_data, tokenizer, MAX_LEN)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to compute and save pooled embeddings
def compute_and_save_embeddings(dataloader, model, device):
    embeddings = []
    labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            pooled_embeddings = mean_pooling(outputs, attention_mask)  # Ensure attention_mask is on the same device
            cls_embeddings = outputs[0][:, 0]
            pooled_embeddings = torch.cat((pooled_embeddings, cls_embeddings), 1)
            
            # pooled_embeddings = pool(outputs.last_hidden_state, attention_mask)  # Ensure attention_mask is on the same device
            # pooled_embeddings = outputs[0][:, 0]
            # pooled_embeddings = average_pool(outputs.last_hidden_state, attention_mask)  # Ensure attention_mask is on the same device
            
            embeddings.append(pooled_embeddings.cpu().numpy())
            if 'targets' in batch:
                labels.append(batch['targets'].cpu().numpy())
    embeddings = np.vstack(embeddings)
    labels = np.hstack(labels) if labels else None
    return embeddings, labels

device = "cuda" if torch.cuda.is_available() else "cpu"
# name = 'e5_instr_en_full'
# name = 'USER-bge-m3_full'
model_name = model_name.replace('/', '_')
train_embeddings, train_labels = compute_and_save_embeddings(train_dataloader, model, device)
np.save(f'train_embeddings_{model_name}.npy', train_embeddings)
np.save(f'train_labels_{model_name}.npy', train_labels)
test_embeddings, test_labels = compute_and_save_embeddings(test_dataloader, model, device)

# Save embeddings and labels
np.save(f'test_embeddings_{model_name}.npy', test_embeddings)
np.save(f'test_labels_{model_name}.npy', test_labels)

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

100%|██████████| 381/381 [03:25<00:00,  1.85it/s]


(48665,)

In [82]:
# import warnings

# # Suppress specific FutureWarning
# warnings.filterwarnings("ignore", category=FutureWarning, message="torch.cuda.amp.autocast(args...) is deprecated. Please use torch.amp.autocast('cuda', args...) instead.")

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False



# Load embeddings and labels
# train_embeddings_ = np.load('train_embeddings_USER-bge-m3.npy')
# train_embeddings_1 = np.load('train_embeddings.npy')
# train_embeddings_2 = np.load('train_embeddings_sbert_large.npy')
# train_embeddings_3 = np.load('train_embeddings_rosberta.npy')

train_embeddings_combined = np.concatenate((
    # np.load('train_embeddings_e5_instr_en_no_dup.npy'),
    # np.load('train_embeddings_USER-bge-m3.npy'),
    np.load('/kaggle/input/ods-task-2/train_embeddings_USER-bge-m3_full.npy'),
    # np.load('train_embeddings.npy'),
    # np.load('train_embeddings_sbert_large.npy'),
    # np.load('train_embeddings_sbert_large_full.npy'),
    # np.load('train_embeddings_rosberta.npy'),
    # np.load('train_embeddings_rosberta_full.npy'),
    # np.load('train_embeddings_e5_instr_en_full_no_dup.npy'),
    np.load('/kaggle/input/ods-task-2/train_embeddings_e5_instr_en_full.npy'),
    np.load('/kaggle/working/train_embeddings_sismetanin_sbert-ru-sentiment-rureviews.npy'),
), axis=1)

# train_labels_1 = np.load('train_labels.npy', allow_pickle=True)
# train_labels_ = np.load('train_labels_e5_instr_en_full.npy', allow_pickle=True)
# train_labels_ = np.load('train_labels_e5_instr_en_full_no_dup.npy', allow_pickle=True)
# train_embeddings_combined = np.concatenate((
#     train_embeddings_,
#     # train_embeddings_1,
#     # train_embeddings_2,
#     train_embeddings_3,
# ), axis=1)
train_labels_ = train_data.rate.to_numpy()

In [83]:
# train_embeddings_combined = np.concatenate((train_embeddings_combined[:, :2048+1024], 
                                            # train_embeddings_combined[:, -1024:],
                                           # ), 1)
train_embeddings_combined.shape
# train_embeddings_.shape

(48665, 4096)

In [84]:
train_embeddings_combined.shape[0]*0.05

2433.25

In [85]:
train_labels_.shape, min(train_labels_), max(train_labels_)

((48665,), 0, 4)

In [86]:
# plt.hist(train_labels, density='probability', label='train')
# plt.hist(val_labels, density='probability',label='val')
# plt.legend()

In [93]:
import warnings

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

# Set the seed
SEED = 42
set_seed(SEED)
# cutoff=30000
cutoff=-1
train_embeddings = train_embeddings_combined[:cutoff]
# train_embeddings = train_embeddings_[:cutoff]
train_labels = train_labels_[:cutoff]
# val_embeddings = np.load('val_embeddings.npy')
# val_labels = np.load('val_labels.npy')
# test_embeddings = np.load('test_embeddings.npy')
# test_labels = np.load('test_labels.npy', allow_pickle=True)

# Perform stratified split
train_embeddings, val_embeddings, train_labels, val_labels = train_test_split(
    train_embeddings,
    train_labels,
    test_size=0.2,  # Adjust the test_size as needed
    random_state=100,
    stratify=train_labels
)

# Create dataset and dataloader for embeddings
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        set_seed(SEED)
        self.embeddings = embeddings
        self.labels = labels

    def __getitem__(self, index):
        return {
            'embeddings': torch.tensor(self.embeddings[index], dtype=torch.float),
            'labels': torch.tensor(self.labels[index], dtype=torch.long)
        }

    def __len__(self):
        return len(self.embeddings)

train_dataset = EmbeddingDataset(train_embeddings, train_labels)
val_dataset = EmbeddingDataset(val_embeddings, val_labels)
# test_dataset = EmbeddingDataset(test_embeddings, test_labels)
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes, dropout_rate):
        set_seed(SEED)
        super(Classifier, self).__init__()
        linear_size1 = 1024
        linear_size2 = 512
        linear_size3 = 128
        self.dense_1 = nn.Linear(input_dim, linear_size1)
        self.batch_norm_1 = nn.BatchNorm1d(linear_size1)
        self.dense_2 = nn.Linear(linear_size1 + input_dim, linear_size2)
        self.dense_3 = nn.Linear(linear_size2, linear_size3)
        self.batch_norm_2 = nn.BatchNorm1d(linear_size2)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier_1 = nn.Linear(linear_size1, num_classes)
        self.classifier_2 = nn.Linear(linear_size2, num_classes)
        self.classifier_3 = nn.Linear(linear_size3, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input):
        x = self.dense_1(input)
        x = torch.cat((x, input), 1)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.dense_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.dense_3(x)
        x = self.dropout(x)
        # x = self.relu(x)
        
        # x = self.batch_norm1(x)
        # x = self.pre_classifier1(x)
        # x = self.batch_norm2(x)
        # x = self.relu(x)
        x = self.classifier_3(x)
        x = self.relu(x)
        # x = self.dropout(x)
        return x

# Initialize model, optimizer, and loss function
input_dim = train_embeddings.shape[1]
num_classes = 5
dropout_rate = 0.3
model = Classifier(input_dim, num_classes, dropout_rate)
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.device_count() > 1:
    # print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model, device_ids = [0,1])
model.to(device)

class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
# criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.02)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(),
                        # lr=5e-7,
                        lr=1e-6,
                        # weight_decay=5e-2,
                        weight_decay=1e-2,
                       )

# Define the scheduler
# scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Reduce LR by a factor of 0.1 every 10 epochs
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)


# Early stopping parameters
early_stopping_patience = 5
best_val_loss = float('inf')
patience_counter = 0

# Training and validation loop
n_epochs = 100
# n_epochs = 41
history = {'train_loss': [], 'val_loss': [], 'val_f1': [], 'max_val_f1': (0,0)}

for epoch in range(n_epochs):
    model.train()
    train_losses = []
    for batch in train_dataloader:
        embeddings = batch['embeddings'].to(device)
        labels = batch['labels'].to(device)
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    train_loss = np.mean(train_losses)
    
    model.eval()
    val_losses = []
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            embeddings = batch['embeddings'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            val_losses.append(loss.item())
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    val_loss = np.mean(val_losses)
    val_f1 = f1_score(all_labels, all_preds, average='weighted')
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_f1)
    if val_f1 > history['max_val_f1'][0]:
        history['max_val_f1'] = (val_f1, epoch+1)
        
    if epoch%5 ==0:
        print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_model_by_stopping.pth')
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Early stopping triggered")
            break
    
    # Step the scheduler
    scheduler.step(val_loss)

print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}")
print(f"TOP F1 SCORE: {history['max_val_f1'][0]}, epoch={history['max_val_f1'][1]}")
torch.save(model.state_dict(), 'best_model.pth')
# Plot training history
plt.plot(history['train_loss'], label='train_loss')
plt.plot(history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Val Loss Over Epochs')
plt.legend()
plt.show()

plt.plot(history['val_f1'], label='val_f1')
plt.xlabel('Epoch')
plt.ylabel('F1 weighted')
plt.title('Val F1 score Over Epochs')
plt.legend()
plt.show()

# Save the trained model
torch.save(model.state_dict(), 'classifier_model.pth')
# TOP F1 SCORE: 0.679734593296176, epoch=48
# Epoch 41/100, Train Loss: 0.7494, Val Loss: 0.7440, Val F1: 0.6781

# Epoch 1/100, Train Loss: 0.9842, Val Loss: 0.8107, Val F1: 0.6442
# Epoch 6/100, Train Loss: 0.7943, Val Loss: 0.7564, Val F1: 0.6699
# Epoch 11/100, Train Loss: 0.7810, Val Loss: 0.7467, Val F1: 0.6704
# Epoch 16/100, Train Loss: 0.7685, Val Loss: 0.7420, Val F1: 0.6737
# Epoch 21/100, Train Loss: 0.7618, Val Loss: 0.7397, Val F1: 0.6775
# Epoch 26/100, Train Loss: 0.7583, Val Loss: 0.7382, Val F1: 0.6783
# Epoch 31/100, Train Loss: 0.7543, Val Loss: 0.7365, Val F1: 0.6809
# Epoch 36/100, Train Loss: 0.7473, Val Loss: 0.7361, Val F1: 0.6810
# Epoch 41/100, Train Loss: 0.7457, Val Loss: 0.7358, Val F1: 0.6816
# Epoch 46/100, Train Loss: 0.7415, Val Loss: 0.7352, Val F1: 0.6826

Epoch 1/100, Train Loss: 1.4565, Val Loss: 1.1777, Val F1: 0.3786
Epoch 6/100, Train Loss: 0.8492, Val Loss: 0.8196, Val F1: 0.6491
Epoch 11/100, Train Loss: 0.8100, Val Loss: 0.7849, Val F1: 0.6566
Epoch 16/100, Train Loss: 0.7973, Val Loss: 0.7712, Val F1: 0.6635
Epoch 21/100, Train Loss: 0.7877, Val Loss: 0.7641, Val F1: 0.6669
Epoch 26/100, Train Loss: 0.7791, Val Loss: 0.7589, Val F1: 0.6670
Epoch 31/100, Train Loss: 0.7752, Val Loss: 0.7550, Val F1: 0.6695


KeyboardInterrupt: 

In [245]:
# import torch
# import numpy as np
# from torch.utils.data import DataLoader, Dataset
# from sklearn.metrics import f1_score
# from sklearn.utils.class_weight import compute_class_weight
# from sklearn.model_selection import StratifiedKFold
# import matplotlib.pyplot as plt

# def set_seed(seed):
#     random.seed(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False

# class EmbeddingDataset(Dataset):
#     def __init__(self, embeddings, labels):
#         self.embeddings = embeddings
#         self.labels = labels

#     def __getitem__(self, index):
#         return {
#             'embeddings': torch.tensor(self.embeddings[index], dtype=torch.float),
#             'labels': torch.tensor(self.labels[index], dtype=torch.long)
#         }

#     def __len__(self):
#         return len(self.embeddings)

# class Classifier(nn.Module):
#     def __init__(self, input_dim, num_classes, dropout_rate):
#         super(Classifier, self).__init__()
#         linear_size1 = 1024
#         linear_size2 = 256
#         self.pre_classifier = nn.Linear(input_dim, linear_size1)
#         self.batch_norm1 = nn.BatchNorm1d(linear_size1)
#         self.pre_classifier1 = nn.Linear(linear_size1, linear_size2)
#         self.batch_norm2 = nn.BatchNorm1d(linear_size2)
#         self.dropout = nn.Dropout(dropout_rate)
#         self.classifier1 = nn.Linear(linear_size1, num_classes)
#         self.classifier2 = nn.Linear(linear_size2, num_classes)
#         self.relu = nn.ReLU()

#     def forward(self, x):
#         x = self.pre_classifier(x)
#         x = self.batch_norm1(x)
#         x = self.relu(x)
#         x = self.dropout(x)
#         x = self.classifier1(x)
#         return x

# def train_and_validate(train_embeddings, train_labels, val_embeddings, val_labels, input_dim, num_classes, dropout_rate, batch_size, n_epochs, device):
#     train_dataset = EmbeddingDataset(train_embeddings, train_labels)
#     val_dataset = EmbeddingDataset(val_embeddings, val_labels)
#     train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#     model = Classifier(input_dim, num_classes, dropout_rate)
#     if torch.cuda.device_count() > 1:
#         model = torch.nn.DataParallel(model, device_ids=[0, 1])
#     model.to(device)

#     class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
#     class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
#     criterion = nn.CrossEntropyLoss(weight=class_weights)
#     optimizer = optim.AdamW(model.parameters(), lr=2e-6, weight_decay=1e-2)
#     scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

#     early_stopping_patience = 5
#     best_val_loss = float('inf')
#     patience_counter = 0

#     history = {'train_loss': [], 'val_loss': [], 'val_f1': [], 'max_val_f1': (0, 0)}

#     for epoch in range(n_epochs):
#         model.train()
#         train_losses = []
#         for batch in train_dataloader:
#             embeddings = batch['embeddings'].to(device)
#             labels = batch['labels'].to(device)
#             optimizer.zero_grad()
#             outputs = model(embeddings)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             train_losses.append(loss.item())
#         train_loss = np.mean(train_losses)

#         model.eval()
#         val_losses = []
#         all_preds = []
#         all_labels = []
#         with torch.no_grad():
#             for batch in val_dataloader:
#                 embeddings = batch['embeddings'].to(device)
#                 labels = batch['labels'].to(device)
#                 outputs = model(embeddings)
#                 loss = criterion(outputs, labels)
#                 val_losses.append(loss.item())
#                 preds = outputs.argmax(dim=1)
#                 all_preds.extend(preds.cpu().numpy())
#                 all_labels.extend(labels.cpu().numpy())
#         val_loss = np.mean(val_losses)
#         val_f1 = f1_score(all_labels, all_preds, average='weighted')

#         history['train_loss'].append(train_loss)
#         history['val_loss'].append(val_loss)
#         history['val_f1'].append(val_f1)
#         if val_f1 > history['max_val_f1'][0]:
#             history['max_val_f1'] = (val_f1, epoch + 1)

#         if epoch % 5 == 0:
#             print(f"Epoch {epoch + 1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val F1: {val_f1:.4f}")

#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             patience_counter = 0
#             torch.save(model.state_dict(), 'best_model.pth')
#         else:
#             patience_counter += 1
#             if patience_counter >= early_stopping_patience:
#                 print("Early stopping triggered")
#                 break

#         scheduler.step(val_loss)

#     print(f"TOP F1 SCORE: {history['max_val_f1'][0]}, epoch={history['max_val_f1'][1]}")
#     return history

# def plot_history(history):
#     plt.plot(history['train_loss'], label='train_loss')
#     plt.plot(history['val_loss'], label='val_loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.title('Train and Val Loss Over Epochs')
#     plt.legend()
#     plt.show()

#     plt.plot(history['val_f1'], label='val_f1')
#     plt.xlabel('Epoch')
#     plt.ylabel('F1 weighted')
#     plt.title('Val F1 score Over Epochs')
#     plt.legend()
#     plt.show()


# from sklearn.model_selection import StratifiedKFold

# # Set the seed
# SEED = 42
# set_seed(SEED)

# # Load embeddings and labels
# train_embeddings = train_embeddings_combined
# train_labels = train_labels_

# # Define parameters
# input_dim = train_embeddings.shape[1]
# num_classes = 5
# dropout_rate = 0.5
# batch_size = 128
# n_epochs = 20
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Stratified K-Fold Cross-Validation
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
# fold = 1

# for train_index, val_index in kf.split(train_embeddings, train_labels):
#     print(f"Fold {fold}")
#     train_embeddings_fold, val_embeddings_fold = train_embeddings[train_index], train_embeddings[val_index]
#     train_labels_fold, val_labels_fold = train_labels[train_index], train_labels[val_index]

#     history = train_and_validate(train_embeddings_fold, train_labels_fold, val_embeddings_fold, val_labels_fold,
#                                  input_dim, num_classes, dropout_rate, batch_size, n_epochs, device)
#     plot_history(history)
#     fold += 1

In [279]:
test_embeddings_combined = np.concatenate((
    # np.load('train_embeddings_e5_instr_en_no_dup.npy'),
    np.load('test_embeddings_USER-bge-m3_full.npy'),
    # np.load('train_embeddings.npy'),
    # np.load('test_embeddings_sbert_large.npy'),
    np.load('test_embeddings_rosberta_full.npy'),
    np.load('test_embeddings_e5_instr_en_full.npy'),
), axis=1)

test_labels = torch.zeros(test_embeddings_combined.shape[0], dtype=torch.long)

In [280]:
test_embeddings_combined.shape, test_labels.shape

((12167, 3072), torch.Size([12167]))

In [281]:
input_dim = test_embeddings_combined.shape[1]

model = Classifier(input_dim, num_classes, dropout_rate)
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model, device_ids=[0, 1])
model.to(device)

# Load the saved model weights
model.load_state_dict(torch.load('best_model.pth'))

# Predict on test data
model.eval()
all_preds = []
all_labels = []

test_dataset = EmbeddingDataset(test_embeddings_combined, test_labels)
batch_size = 128
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

with torch.no_grad():
    for batch in test_dataloader:
    # for batch in val_dataloader:
        embeddings = batch['embeddings'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(embeddings)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# # Calculate F1 score
from sklearn.metrics import f1_score
test_f1 = f1_score(all_labels, all_preds, average='weighted')
print(f"Test F1 score: {test_f1:.5f}")

# Optionally, save the predictions
np.save('test_predictions.npy', all_preds)

/tmp/ipykernel_30/876705421.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'labels': torch.tensor(self.labels[index], dtype=torch.long)


Test F1 score: 0.18741


In [282]:
max(all_preds), min(all_preds)

(4, 0)

In [283]:
sample_submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))

sample_submission["rate"] = all_preds

sample_submission.rate = le.inverse_transform(sample_submission.rate)

print(sample_submission.describe())

sample_submission.to_csv("submission_06797_1_percent_with_dups_41_ep_new.csv", index=False)

              index          rate
count  12167.000000  12167.000000
mean    6083.000000      4.135202
std     3512.454697      1.294172
min        0.000000      1.000000
25%     3041.500000      4.000000
50%     6083.000000      5.000000
75%     9124.500000      5.000000
max    12166.000000      5.000000


In [149]:
train_split, val_split = train_test_split(
    # train_data,
    # unique_train,
    train_data_no_duplicates,
    # test_size=0.1,
    test_size=0.95,
    random_state=100,
    # stratify=train_data.rate
    # stratify=unique_train.rate
    stratify=train_data_no_duplicates.rate
    )


# train_dataset = FiveDataset(train_split, tokenizer, MAX_LEN)

# val_dataset = FiveDataset(val_split, tokenizer, MAX_LEN)



experiment_name = "unique train 90% e5 cleaned dups weighed loss drop 0.3"



model_config = {

    "num_classes": 5,

    "dropout_rate": 0.1,

    # "freeze_bert": True,

    "freeze_bert": True,

}

# model_name = "DeepPavlov/rubert-base-cased"
# model_name = "sismetanin/rubert-ru-sentiment-rureviews"
model_name="d0rj/e5-large-en-ru"



trainer_config = {

    # "lr": 5e-6,
    "lr": 1e-3,

    "n_epochs": 10,

    # "weight_decay": 1e-2,
    "weight_decay": 1e-4,

    # "batch_size": BATCH_SIZE,
    # "batch_size": 32,
    "batch_size": 128,

    "device": "cuda" if torch.cuda.is_available() else "cpu",

    "seed": 100,
    "automodel": False,
    "is_bert": False

    

    # "verbose": False

}


# ep 5, lr 1e-5
# Train Loss, mean and std: 0.62, 0.0739
# Val loss: 0.8456820249557495
# Val F1 score: 0.675

# ep 5, lr 5e-6
# Train Loss, mean and std: 0.701, 0.0746
# Val loss: 0.8005812168121338
# Val F1 score: 0.673

# Train Loss, mean and std: 1.2, 0.141
# Val loss: 1.0960309505462646
# Val F1 score: 0.657
# Epoch 2/10

#   0%|          | 0/330 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
#   warnings.warn(
# /opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
#   with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
# Loss=1.09: 100%|██████████| 330/330 [07:48<00:00,  1.42s/it] 
# 100%|██████████| 37/37 [00:18<00:00,  2.02it/s]

# Train Loss, mean and std: 1.06, 0.0821
# Val loss: 1.070529580116272
# Val F1 score: 0.645
# Epoch 3/10



trainer, test_dataloader = train_and_validate(model_name, trainer_config, model_config, experiment_name)

train size  2276
UNIQUE TRAIN 90% E5 CLEANED DUPS WEIGHED LOSS DROP 0.3, LR 0.001, WD 0.0001, EP 10, BATCH 128
------------------------------
Epoch 1/10


  0%|          | 0/18 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/tmp/ipykernel_30/307525309.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(ids, dtype=torch.long),
/tmp/ipykernel_30/307525309.py:72: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor

AttributeError: Caught AttributeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 83, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1553, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1562, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_30/351603053.py", line 74, in forward
    input_ids=batch_dict.get(input_ids),
AttributeError: 'NoneType' object has no attribute 'get'


In [51]:
train_split, val_split = train_test_split(
    # train_data,
    # unique_train,
    train_data_no_duplicates,
    test_size=0.1,
    # test_size=0.95,
    random_state=100,
    # stratify=train_data.rate
    # stratify=unique_train.rate
    stratify=train_data_no_duplicates.rate
    )


# train_dataset = FiveDataset(train_split, tokenizer, MAX_LEN)

# val_dataset = FiveDataset(val_split, tokenizer, MAX_LEN)



experiment_name = "unique train 90% rubert rureviews cleaned dups weighed loss drop 0.3"



model_config = {

    "num_classes": 5,

    "dropout_rate": 0.3,

    # "freeze_bert": True,

    "freeze_bert": False,

}

# model_name = "DeepPavlov/rubert-base-cased"
model_name = "sismetanin/rubert-ru-sentiment-rureviews"



trainer_config = {

    # "lr": 5e-6,
    "lr": 1e-5,

    "n_epochs": 10,

    # "weight_decay": 1e-2,
    "weight_decay": 1e-4,

    # "batch_size": BATCH_SIZE,
    # "batch_size": 32,
    "batch_size": 128,

    "device": "cuda" if torch.cuda.is_available() else "cpu",

    "seed": 100,

    

    # "verbose": False

}


# ep 5, lr 1e-5
# Train Loss, mean and std: 0.62, 0.0739
# Val loss: 0.8456820249557495
# Val F1 score: 0.675

# ep 5, lr 5e-6
# Train Loss, mean and std: 0.701, 0.0746
# Val loss: 0.8005812168121338
# Val F1 score: 0.673

# Train Loss, mean and std: 1.2, 0.141
# Val loss: 1.0960309505462646
# Val F1 score: 0.657
# Epoch 2/10

#   0%|          | 0/330 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
#   warnings.warn(
# /opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
#   with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
# Loss=1.09: 100%|██████████| 330/330 [07:48<00:00,  1.42s/it] 
# 100%|██████████| 37/37 [00:18<00:00,  2.02it/s]

# Train Loss, mean and std: 1.06, 0.0821
# Val loss: 1.070529580116272
# Val F1 score: 0.645
# Epoch 3/10



trainer, test_dataloader = train_and_validate(model_name, trainer_config, model_config, experiment_name)

train size         rate                                               text  \
36122     4                                        Тут хорошо    
22223     3  Удобно расположены прилавки. Доброжелательный ...   
24380     2            Маленький магазин маленький ассортимент   
30004     4                             Всегда хожу все ништяк   
34223     4  Большой плюс это кассы самообслуживания, а так...   
...     ...                                                ...   
9407      4        На редкость приятный коллектив сотрудников.   
838       3  Ассортимент стал маленький цены прыгают то дос...   
41559     4                       Любимый магазин. Живу рядом.   
35210     0                                    Цены заоблочные   
9538      2  В целом, обычная Пятерочка - цены завышены, ас...   

                                            cleaned_text  
36122                                        тут хорошо   
22223  удобно расположены прилавки доброжелательный и...  
24380            м

  0%|          | 0/321 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 36/36 [00:17<00:00,  2.09it/s]


Train Loss, mean and std: 1.16, 0.124
Val loss: 1.077704906463623
Val F1 score: 0.641
Epoch 2/10


  0%|          | 0/321 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 36/36 [00:17<00:00,  2.08it/s]


Train Loss, mean and std: 1.03, 0.0937
Val loss: 1.0746452808380127
Val F1 score: 0.646
Epoch 3/10


  0%|          | 0/321 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
100%|██████████| 36/36 [00:17<00:00,  2.08it/s]


Train Loss, mean and std: 0.962, 0.09
Val loss: 1.0807594060897827
Val F1 score: 0.644
Epoch 4/10


  0%|          | 0/321 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Loss=1.07:  51%|█████▏    | 165/321 [03:52<03:40,  1.41s/it] 


KeyboardInterrupt: 

In [48]:
train_split, val_split = train_test_split(
    # train_data,
    # unique_train,
    train_data_no_duplicates,
    test_size=0.1,
    # test_size=0.95,
    random_state=100,
    # stratify=train_data.rate
    # stratify=unique_train.rate
    stratify=train_data_no_duplicates.rate
    )

experiment_name = "unique train 90% ruBert-base weighed loss drop 0.3"

model_config = {
    "num_classes": 5,
    "dropout_rate": 0.3,
    # "freeze_bert": True,
    "freeze_bert": False,
}

# model_name = "DeepPavlov/rubert-base-cased"
# model_name = "sismetanin/rubert-ru-sentiment-rureviews"
model_name = "ai-forever/ruBert-base"



trainer_config = {

    # "lr": 5e-6,
    # "lr": 1e-5,
    "lr": 2e-6,
    "n_epochs": 10,
    "weight_decay": 1e-2,
    # "weight_decay": 1e-4,
    # "batch_size": BATCH_SIZE,
    "batch_size": 128,
    # "batch_size": 128,
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "seed": 100,
    # "verbose": False
}

# RUREVIEWS
# ep 5, lr 1e-5 
# Train Loss, mean and std: 0.62, 0.0739
# Val loss: 0.8456820249557495
# Val F1 score: 0.675

# ep 5, lr 5e-6
# Train Loss, mean and std: 0.701, 0.0746
# Val loss: 0.8005812168121338
# Val F1 score: 0.673

# rubert base ep 4 lr 2e-6
# Train Loss, mean and std: 1.01, 0.113
# Val loss: 1.0344709157943726
# Val F1 score: 0.671
# Epoch 5/10


trainer, test_dataloader = train_and_validate(model_name, trainer_config, model_config, experiment_name)

UNIQUE TRAIN 90% RUBERT-BASE WEIGHED LOSS DROP 0.3, LR 2E-06, WD 0.01, EP 10, BATCH 128
------------------------------
Epoch 1/10


  0%|          | 0/321 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2837: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Loss=1.6:   5%|▍         | 16/321 [00:24<07:47,  1.53s/it] 


KeyboardInterrupt: 

# Get testset predictions


In [ ]:
# predictions = trainer.predict(test_dataloader)

# sample_submission = pd.read_csv(os.path.join(PATH, "sample_submission.csv"))

# sample_submission["rate"] = predictions

# sample_submission.rate = le.inverse_transform(sample_submission.rate)

# sample_submission.head()

# sample_submission.to_csv("submission_best_transformer.csv", index=False)